#Податочно множество
Симнете го датасетот од линкот кој ви е поставен на самата задача, потоа истиот поставете го локално за да го вчитате.
Во случај на грешка при вчитување на .csv додадете аргумент `encoding='latin-1`


In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install transformers[torch] datasets evaluate

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Input, Dense, LSTM, Embedding, concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Претпроцесирање кое е потребно

In [ ]:
data = pd.read_csv('/content/text_sentiment.csv')

In [ ]:
data.isnull().sum()

text         0
Sentiment    0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
data['Sentiment'] = encoder.fit_transform(data['Sentiment'])

In [ ]:
data.head(4)

,text,Sentiment
0,Awe-inspired by the grandeur of an ancient cat...,17
1,Awe-struck by the grandeur of an ancient cathe...,17
2,"Walking the Great Wall of China, each step a t...",17
3,Euphoria floods in as the final puzzle piece c...,75


In [ ]:
len(data['Sentiment'].unique())

191

In [ ]:
data.rename(columns={'Sentiment': 'label'}, inplace=True)

In [ ]:
X, Y = data['text'], data['label']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [ ]:
train_data = pd.concat([X_train, Y_train], axis=1)

In [ ]:
test_data = pd.concat([X_test, Y_test], axis=1)

In [ ]:
train_data.to_csv('train.csv')
test_data.to_csv('test.csv')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={'train': '/content/train.csv', 'test': '/content/test.csv'})
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 585
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 147
    })
})

In [ ]:
dataset['train'] = dataset['train'].remove_columns(column_names='Unnamed: 0')
dataset['test'] = dataset['test'].remove_columns(column_names='Unnamed: 0')

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 585
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 147
    })
})

#Модел 1: Трансформер модел

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("rizwan-ai/distilbert-base-uncased-finetuned-emotion")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("rizwan-ai/distilbert-base-uncased-finetuned-emotion", num_labels=191)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=5e-5
)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    metric_for_best_model="f1",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

In [ ]:
trainer.train()

IndexError: Target 146 is out of bounds.

#Модел 2: Невронска мрежа

#Eвалуација на двата модели и споредба